Sample Ann

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import re

In [ ]:
tf.__version__

Load data

In [ ]:
!curl -L -o DeviceAbuseDataFall.zip https://github.com/arnabalumnus/Zebra/raw/filter_data_chunk/python/DeviceAbuseDataFall.zip

In [ ]:
import zipfile
with zipfile.ZipFile('DeviceAbuseDataFall.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
# Create a folder with name `DeviceAbuseDataFall`
# Copy all data .csv file into this folder before running

def preProcess(rowCount):
    trainingDataFolder = "DeviceAbuseDataFall/"

    dirPath = os.path.join(os.getcwd(), trainingDataFolder)
    files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]

    fileList = []
    finalDF = pd.DataFrame()
    
    for file in files:
        # Importing the dataset
        orgData = pd.read_csv(dirPath + file)
        data = pd.DataFrame(orgData.iloc[:, 1:4])

        print('Current File :', file, '# Rowcount :', data.shape[0])       
        if data.shape[0] > rowCount:
            fileList.append(file)
            start = 0
            end = rowCount
           
            count = 0
            for i in range(start, end):
                temp = pd.DataFrame(data.iloc[i:i+1, 0:3])
                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        elif (data.shape[0] >= (rowCount / 2)):
            fileList.append(file)
            diff = rowCount - data.shape[0]
            start = 0
            end = rowCount
            count = 0
            countDown = rowCount - data.shape[0]

            for i in range(start, end):
                if (countDown > 0):
                    temp = pd.DataFrame(data.iloc[0:1, 0:3])
                    countDown -= 1
                else:
                    temp = pd.DataFrame(data.iloc[(i - diff):(i + 1 - diff), 0:3])

                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        else:
            print("No. of rows is less than", (rowCount / 2))
           
    fileList = [i.split('_', 1)[0] for i in fileList]
    fileList = [re.sub("\d+", "", i) for i in fileList]
    actualOutput = pd.DataFrame([fileName[:4] for fileName in fileList])
    actualOutput.columns = ["actualOutput"]
    
    finalDF = finalDF.reset_index(drop=True)
    finalDF = pd.concat([finalDF, actualOutput], axis=1)
    
    return finalDF

In [ ]:
finalDF = preProcess(200)

In [9]:
print(finalDF)

           X0        Y0         Z0  ...      Y199       Z199  actualOutput
0   -0.460602  0.460602  10.210007  ...  0.537369  10.286774          FFZM
1    0.383835  0.383835   9.442337  ...  0.614136  10.286774          HFYP
2    9.212036  0.153534   0.997971  ...  0.076767   0.921204          FFXP
3    0.076767  0.230301  10.286774  ...  0.307068  10.056473          FFZM
4    0.383835  3.684814   8.981735  ...  5.987824   6.909027          UTXX
..        ...       ...        ...  ...       ...        ...           ...
340  9.288803  0.537369   0.844437  ...  0.460602   0.537369          FFXP
341  0.460602  5.680756   7.676697  ...  7.216095   8.981735          UTXX
342 -0.844437  5.834290   9.672638  ... -2.763611   2.993912          OTXX
343  0.000000  0.537369   9.979706  ...  0.690903  10.210007          HFYP
344 -0.230301  0.230301  10.363541  ...  0.230301  10.210007          HFXP

[345 rows x 601 columns]


In [10]:
X=finalDF.iloc[:, :-1].values

In [11]:
print(X)

[[ -0.4606018    0.4606018   10.210007   ...   0.15353394   0.5373688
   10.286774  ]
 [  0.38383484   0.38383484   9.442337   ...  -0.07676697   0.61413574
   10.286774  ]
 [  9.212036     0.15353394   0.9979706  ...   9.135269     0.07676697
    0.9212036 ]
 ...
 [ -0.84443665   5.8342896    9.672638   ... -12.896851    -2.7636108
    2.9939117 ]
 [  0.           0.5373688    9.979706   ...  -0.07676697   0.6909027
   10.210007  ]
 [ -0.2303009    0.2303009   10.363541   ...  -0.30706787   0.2303009
   10.210007  ]]


In [12]:
y=finalDF.iloc[:, -1].values

In [13]:
print(y)

['FFZM' 'HFYP' 'FFXP' 'FFZM' 'UTXX' 'PFXX' 'FFXP' 'HFYP' 'HFXP' 'HFYP'
 'FFZM' 'UTXX' 'FFYP' 'HFXP' 'FFYP' 'HFXP' 'PFXX' 'HFYP' 'PFXX' 'PFXX'
 'HFXP' 'PFXX' 'FFYP' 'FFZM' 'FFZM' 'OTXX' 'OTXX' 'FFXP' 'FFXP' 'UTXX'
 'HFXP' 'FFZM' 'FFXP' 'PFXX' 'FFZM' 'FFXP' 'FFZM' 'FFXP' 'FFXP' 'UTXX'
 'FFXP' 'FFXP' 'FFYP' 'UTXX' 'FFXP' 'HFYP' 'FFZM' 'FFXP' 'UTXX' 'FFZM'
 'FFXP' 'HFYP' 'FFXP' 'HFXP' 'PFXX' 'UTXX' 'PFXX' 'PFXX' 'FFXP' 'HFYP'
 'PFXX' 'HFYP' 'UTXX' 'FFZM' 'FFXP' 'FFZM' 'FFXP' 'UTXX' 'UTXX' 'HFXP'
 'FFZM' 'OTXX' 'HFXP' 'FFZM' 'HFYP' 'FFXP' 'FFXP' 'FFZM' 'HFYP' 'FFXP'
 'HFXP' 'FFZM' 'FFZM' 'FFZM' 'FFXP' 'FFXP' 'FFYP' 'FFXP' 'FFZM' 'FFZM'
 'PFXX' 'HFXP' 'HFXP' 'FFZM' 'UTXX' 'HFXP' 'FFXP' 'FFYP' 'HFYP' 'FFXP'
 'FFZM' 'FFXP' 'FFZM' 'FFZM' 'PFXX' 'HFXP' 'FFXP' 'UTXX' 'FFZM' 'FFXP'
 'FFYP' 'HFYP' 'FFXP' 'FFZM' 'FFZM' 'FFZM' 'FFZM' 'HFYP' 'FFZM' 'FFYP'
 'PFXX' 'HFXP' 'PFXX' 'HFXP' 'OTXX' 'FFZM' 'FFXP' 'FFZM' 'UTXX' 'FFXP'
 'FFZM' 'UTXX' 'HFYP' 'UTXX' 'HFYP' 'HFXP' 'FFYP' 'FFXP' 'FFZM' 'HFYP'
 'UTXX

In [14]:
type(y)

numpy.ndarray

In [15]:
myset = set(y)
print(myset)

{'FFYP', 'FFXP', 'PFXX', 'HFXP', 'FFZM', 'UTXX', 'HFYP', 'OTXX'}


In [16]:
outcomeDict = { 'HFYP': 0, 'PFXX' : 1, 'FFXP' : 2, 'FFZM' : 3, 'HFXP' : 4, 'FFYP': 5, 'OTXX' : 6 , 'UTXX' : 7}

In [17]:
print(y)

['FFZM' 'HFYP' 'FFXP' 'FFZM' 'UTXX' 'PFXX' 'FFXP' 'HFYP' 'HFXP' 'HFYP'
 'FFZM' 'UTXX' 'FFYP' 'HFXP' 'FFYP' 'HFXP' 'PFXX' 'HFYP' 'PFXX' 'PFXX'
 'HFXP' 'PFXX' 'FFYP' 'FFZM' 'FFZM' 'OTXX' 'OTXX' 'FFXP' 'FFXP' 'UTXX'
 'HFXP' 'FFZM' 'FFXP' 'PFXX' 'FFZM' 'FFXP' 'FFZM' 'FFXP' 'FFXP' 'UTXX'
 'FFXP' 'FFXP' 'FFYP' 'UTXX' 'FFXP' 'HFYP' 'FFZM' 'FFXP' 'UTXX' 'FFZM'
 'FFXP' 'HFYP' 'FFXP' 'HFXP' 'PFXX' 'UTXX' 'PFXX' 'PFXX' 'FFXP' 'HFYP'
 'PFXX' 'HFYP' 'UTXX' 'FFZM' 'FFXP' 'FFZM' 'FFXP' 'UTXX' 'UTXX' 'HFXP'
 'FFZM' 'OTXX' 'HFXP' 'FFZM' 'HFYP' 'FFXP' 'FFXP' 'FFZM' 'HFYP' 'FFXP'
 'HFXP' 'FFZM' 'FFZM' 'FFZM' 'FFXP' 'FFXP' 'FFYP' 'FFXP' 'FFZM' 'FFZM'
 'PFXX' 'HFXP' 'HFXP' 'FFZM' 'UTXX' 'HFXP' 'FFXP' 'FFYP' 'HFYP' 'FFXP'
 'FFZM' 'FFXP' 'FFZM' 'FFZM' 'PFXX' 'HFXP' 'FFXP' 'UTXX' 'FFZM' 'FFXP'
 'FFYP' 'HFYP' 'FFXP' 'FFZM' 'FFZM' 'FFZM' 'FFZM' 'HFYP' 'FFZM' 'FFYP'
 'PFXX' 'HFXP' 'PFXX' 'HFXP' 'OTXX' 'FFZM' 'FFXP' 'FFZM' 'UTXX' 'FFXP'
 'FFZM' 'UTXX' 'HFYP' 'UTXX' 'HFYP' 'HFXP' 'FFYP' 'FFXP' 'FFZM' 'HFYP'
 'UTXX

In [18]:
y[y=='HFYP'] = 0
y[y=='PFXX'] = 1
y[y=='FFXP'] = 2
y[y=='FFZM'] = 3
y[y=='HFXP'] = 4
y[y=='FFYP'] = 5
y[y=='OTXX'] = 6
y[y=='UTXX'] = 7

In [19]:
print(y)

[3 0 2 3 7 1 2 0 4 0 3 7 5 4 5 4 1 0 1 1 4 1 5 3 3 6 6 2 2 7 4 3 2 1 3 2 3
 2 2 7 2 2 5 7 2 0 3 2 7 3 2 0 2 4 1 7 1 1 2 0 1 0 7 3 2 3 2 7 7 4 3 6 4 3
 0 2 2 3 0 2 4 3 3 3 2 2 5 2 3 3 1 4 4 3 7 4 2 5 0 2 3 2 3 3 1 4 2 7 3 2 5
 0 2 3 3 3 3 0 3 5 1 4 1 4 6 3 2 3 7 2 3 7 0 7 0 4 5 2 3 0 7 2 3 3 2 1 3 2
 4 3 5 2 3 1 5 1 4 3 0 4 7 1 5 7 3 3 5 3 4 5 5 4 1 2 3 7 3 7 2 7 5 6 0 2 0
 4 0 2 3 4 4 2 3 2 7 6 7 1 4 5 1 2 6 3 3 7 4 2 3 2 0 2 1 3 2 4 1 2 4 1 4 6
 2 1 1 7 5 1 2 2 0 1 4 2 1 3 1 3 1 4 2 7 3 3 3 1 0 5 0 7 0 3 0 3 4 3 7 3 4
 2 1 2 4 3 4 1 1 4 3 3 4 2 0 0 3 5 0 6 2 7 2 0 3 3 3 0 3 7 1 0 1 2 4 6 4 3
 3 2 6 3 0 0 2 2 2 2 7 0 4 3 5 2 0 6 6 4 2 3 7 7 7 2 2 1 1 0 6 0 3 6 3 2 1
 3 3 6 2 2 0 3 2 7 6 0 4]


In [20]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
#encoded_Y = encoder.transform(y)

# convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)


LabelEncoder()

In [21]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(y)

In [22]:
print(dummy_y)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Splitting the dataset into the Training set and Test set


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.2, random_state = 0)

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
print(X_train)

[[-0.31615795 -0.78753672  0.77982606 ... -0.34994589 -0.30349905
  -0.96337717]
 [-0.3979766  -0.56172105  0.80043623 ... -0.12301549 -0.16998655
   0.86349851]
 [-0.009338    0.94371686  0.01724984 ... -2.39231946 -0.70403655
  -1.08219836]
 ...
 [-0.70479656 -0.71226483  0.75921588 ... -0.33248971 -0.41030905
   0.71497198]
 [ 1.95430967 -0.83771798 -1.40485172 ...  1.84953331 -0.27679656
  -0.68117687]
 [-0.35706726  0.86844494  0.4088429  ... -0.05319076  1.35205591
   0.87835117]]


In [26]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [27]:
ann = tf.keras.models.Sequential()

In [28]:
ann.add(tf.keras.layers.Dense(units=1200, activation='relu'))

In [29]:
ann.add(tf.keras.layers.Dense(units=8, activation='softmax'))

In [30]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [31]:
ann.fit(X_train, y_train, batch_size = 5, epochs = 100)

Epoch 1/100
56/56 [==============================] - 1s 5ms/step - loss: 1.3924 - accuracy: 0.6087
Epoch 2/100
56/56 [==============================] - 0s 5ms/step - loss: 0.8325 - accuracy: 0.6884
Epoch 3/100
56/56 [==============================] - 0s 5ms/step - loss: 0.6051 - accuracy: 0.7681
Epoch 4/100
56/56 [==============================] - 0s 4ms/step - loss: 0.6599 - accuracy: 0.7283
Epoch 5/100
56/56 [==============================] - 0s 4ms/step - loss: 0.5390 - accuracy: 0.7826
Epoch 6/100
56/56 [==============================] - 0s 4ms/step - loss: 0.5608 - accuracy: 0.7645
Epoch 7/100
56/56 [==============================] - 0s 5ms/step - loss: 0.5027 - accuracy: 0.7754
Epoch 8/100
56/56 [==============================] - 0s 5ms/step - loss: 0.4602 - accuracy: 0.7790
Epoch 9/100
56/56 [==============================] - 0s 4ms/step - loss: 0.4476 - accuracy: 0.7971
Epoch 10/100
56/56 [==============================] - 0s 5ms/step - loss: 0.4340 - accuracy: 0.7790
Epoch 11/

In [32]:
print(X_test)

[[-0.52070458 -0.43626791  0.8828768  ... -0.64670104 -0.38360655
  -0.99308247]
 [ 1.91340032 -0.58681168 -1.75522454 ...  1.83207712 -0.22339156
  -0.93367189]
 [-0.80706986  1.32007625  0.28518185 ... -0.41977064  1.56567589
   0.75952994]
 ...
 [-0.35706726  0.94371686  0.45006325 ... -0.26266497  1.69918837
   0.58129809]
 [-0.68434188 -0.58681168  1.02714789 ... -0.38485826 -0.59722654
  -0.97822983]
 [-0.62297789 -0.63699294  0.73860571 ... -0.31503353 -0.89095405
  -0.97822983]]


In [33]:
y_pred = ann.predict(X_test)


In [34]:
print(y_pred)

[[2.79543502e-03 1.25670869e-08 3.81841447e-09 9.97154355e-01
  3.92844548e-08 2.18605617e-14 3.48567801e-06 4.67626996e-05]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.32291354e-04 5.25892191e-02 3.30535405e-10 5.02423936e-05
  4.52207327e-10 3.81263963e-14 9.09978807e-01 3.70494314e-02]
 [1.94136737e-05 1.33458425e-10 4.94900387e-09 9.99978185e-01
  2.30182877e-06 8.61135368e-14 2.01061545e-09 1.17056700e-07]
 [1.91543194e-07 6.43811014e-04 5.35133351e-15 4.98949362e-08
  1.72769608e-13 4.66913820e-22 1.51783333e-05 9.99340832e-01]
 [7.01151073e-01 1.13767128e-04 7.23801463e-08 1.83632344e-01
  1.14764862e-01 1.85274371e-06 2.90185300e-04 4.58288123e-05]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]


In [35]:
index = tf.argmax(y_pred, axis=1)

In [36]:
index

<tf.Tensor: shape=(69,), dtype=int64, numpy=
array([3, 2, 6, 3, 7, 0, 2, 2, 0, 3, 2, 7, 0, 0, 2, 2, 2, 4, 1, 4, 7, 7,
       5, 7, 0, 3, 3, 0, 7, 2, 2, 7, 4, 0, 3, 4, 3, 0, 0, 3, 0, 0, 0, 5,
       2, 0, 2, 0, 2, 0, 4, 3, 6, 0, 2, 4, 2, 3, 0, 3, 2, 7, 2, 7, 7, 3,
       7, 4, 3])>

In [37]:
outcome = index.numpy()


In [38]:
outcomeString = encoder.inverse_transform(outcome)

In [39]:
print(outcomeString)

[3 2 6 3 7 0 2 2 0 3 2 7 0 0 2 2 2 4 1 4 7 7 5 7 0 3 3 0 7 2 2 7 4 0 3 4 3
 0 0 3 0 0 0 5 2 0 2 0 2 0 4 3 6 0 2 4 2 3 0 3 2 7 2 7 7 3 7 4 3]


In [40]:
print(index)

tf.Tensor(
[3 2 6 3 7 0 2 2 0 3 2 7 0 0 2 2 2 4 1 4 7 7 5 7 0 3 3 0 7 2 2 7 4 0 3 4 3
 0 0 3 0 0 0 5 2 0 2 0 2 0 4 3 6 0 2 4 2 3 0 3 2 7 2 7 7 3 7 4 3], shape=(69,), dtype=int64)


In [41]:
type(index)

tensorflow.python.framework.ops.EagerTensor

In [42]:
#index.

Read from test data set 

In [43]:
testDataFolder = "DeviceAbuseDataFallTest"
testDataFile = "FF03YPU_2017_08_17_15_40_36  - prefall.csv"
dirPath = os.path.join(os.getcwd(), testDataFolder)

In [44]:
print(dirPath)

/content/DeviceAbuseDataFallTest


In [45]:
testDataFileWithPath = os.path.join(dirPath, testDataFile)

In [46]:
print(testDataFileWithPath)

/content/DeviceAbuseDataFallTest/FF03YPU_2017_08_17_15_40_36  - prefall.csv


In [47]:
testDF = pd.DataFrame()
testData = pd.read_csv(testDataFileWithPath)

In [48]:
print(testData)

             TS         X          Y         Z
0      80566528 -1.381805  10.056473  1.458572
1      90606848 -1.458572  10.133240  1.458572
2     110778880 -1.458572  10.210007  1.765640
3     100708096 -1.458572  10.210007  1.612106
4     120849664 -1.612106   9.826172  1.381805
..          ...       ...        ...       ...
213  2226054144 -0.460602   0.230301  0.383835
214  2236094464 -0.460602   0.230301  0.383835
215  2246195712 -0.537369   0.230301  0.460602
216  2256236032 -0.460602   0.230301  0.460602
217  2266337280 -0.460602   0.153534  0.383835

[218 rows x 4 columns]


In [49]:
testData1 = pd.DataFrame(testData.iloc[:, 1:4])

In [50]:
print(testData1)

            X          Y         Z
0   -1.381805  10.056473  1.458572
1   -1.458572  10.133240  1.458572
2   -1.458572  10.210007  1.765640
3   -1.458572  10.210007  1.612106
4   -1.612106   9.826172  1.381805
..        ...        ...       ...
213 -0.460602   0.230301  0.383835
214 -0.460602   0.230301  0.383835
215 -0.537369   0.230301  0.460602
216 -0.460602   0.230301  0.460602
217 -0.460602   0.153534  0.383835

[218 rows x 3 columns]


In [51]:
testData2 = pd.DataFrame()
count = 0
for i in range(0, 200):
  temp = pd.DataFrame(testData1.iloc[i:i+1, 0:3])
  temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
  temp = temp.reset_index(drop=True)
               
  if count == 0:
    tempDF = temp
    count = 1
  else:
    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
            
               

In [52]:
print(tempDF)

         X0         Y0        Z0  ...      X199       Y199      Z199
0 -1.381805  10.056473  1.458572  ... -1.305038  10.210007  1.074737

[1 rows x 600 columns]


In [53]:
prediction_result = ann.predict(tempDF)

In [54]:
print(prediction_result)

[[0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 7.080682e-16]]


In [55]:
prediction_Value = tf.argmax(prediction_result, axis=1)

In [56]:
print(prediction_Value)

tf.Tensor([1], shape=(1,), dtype=int64)


In [57]:
converter = tf.lite.TFLiteConverter.from_keras_model(ann)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpx7zj_mab/assets


In [58]:
with open('neural_network_fall_data_model.tflite', 'wb') as f:
  f.write(tflite_model)